<a href="https://colab.research.google.com/github/Matiiwtcp/Proyecto_final_bigdata/blob/main/Proyecto_final_MLBG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

Conectar drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importamos librerias

In [2]:
#import os
import numpy as np
import pandas as pd
import geopandas as gpd
#import matplotlib.pyplot as plt
#import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


Cargamos dataset

In [3]:
ruta_de_archivos = '/content/drive/MyDrive/Global_Education.csv'
dataframe = pd.read_csv(ruta_de_archivos, encoding='latin1')
df = dataframe.copy ()

In [4]:
# Convertir valores nulos a NaN
df.replace('Datos nulos', np.nan, inplace=True)
# Eliminar filas con NaN
df_cleaned = df.dropna()

# Ajustar nombres de columnas eliminando espacios adicionales
df.columns = df.columns.str.strip()

# Diccionario de traducción de nombres de columnas
nuevos_nombres = {
    'Countries and areas': 'Países y Zonas',
    'Latitude': 'Latitud',
    'Longitude': 'Longitud',
    'OOSR_Pre0Primary_Age_Male': 'Tasa No Escolarización Preescolar (Varones)',
    'OOSR_Pre0Primary_Age_Female': 'Tasa No Escolarización Preescolar (Mujeres)',
    'OOSR_Primary_Age_Male': 'Tasa No Escolarización Primaria (Varones)',
    'OOSR_Primary_Age_Female': 'Tasa No Escolarización Primaria (Mujeres)',
    'OOSR_Lower_Secondary_Age_Male': 'Tasa No Escolarización Primer Ciclo Secundaria (Varones)',
    'OOSR_Lower_Secondary_Age_Female': 'Tasa No Escolarización Primer Ciclo Secundaria (Mujeres)',
    'OOSR_Upper_Secondary_Age_Male': 'Tasa No Escolarización Segundo Ciclo Secundaria (Varones)',
    'OOSR_Upper_Secondary_Age_Female': 'Tasa No Escolarización Segundo Ciclo Secundaria (Mujeres)',
    'Completion_Rate_Primary_Male': 'Tasa Finalización Educación Primaria (Varones)',
    'Completion_Rate_Primary_Female': 'Tasa Finalización Educación Primaria (Mujeres)',
    'Completion_Rate_Lower_Secondary_Male': 'Tasa Finalización Primer Ciclo Secundaria (Varones)',
    'Completion_Rate_Lower_Secondary_Female': 'Tasa Finalización Primer Ciclo Secundaria (Mujeres)',
    'Completion_Rate_Upper_Secondary_Male': 'Tasa Finalización Segundo Ciclo Secundaria (Varones)',
    'Completion_Rate_Upper_Secondary_Female': 'Tasa Finalización Segundo Ciclo Secundaria (Mujeres)',
    'Grade_2_3_Proficiency_Reading': 'Competencia Lectura 2º y 3º Grado',
    'Grade_2_3_Proficiency_Math': 'Competencia Matemáticas 2º y 3º Grado',
    'Primary_End_Proficiency_Reading': 'Competencia Lectora Final Educación Primaria',
    'Primary_End_Proficiency_Math': 'Dominio Matemáticas Final Educación Primaria',
    'Lower_Secondary_End_Proficiency_Reading': 'Competencia Lectora Final Primer Ciclo Secundaria',
    'Lower_Secondary_End_Proficiency_Math': 'Dominio Matemáticas Final Primer Ciclo Secundaria',
    'Youth_15_24_Literacy_Rate_Male': 'Tasa Alfabetización Jóvenes (Varones 15-24 años)',
    'Youth_15_24_Literacy_Rate_Female': 'Tasa Alfabetización Jóvenes (Mujeres 15-24 años)',
    'Birth_Rate': 'Tasa Natalidad',
    'Gross_Primary_Education_Enrollment': 'Matrícula Bruta Educación Primaria',
    'Gross_Tertiary_Education_Enrollment': 'Matrícula Bruta Educación Terciaria',
    'Unemployment_Rate': 'Tasa Desempleo'
}


# Renombrar las columnas en el DataFrame
df.rename(columns=nuevos_nombres, inplace=True)

In [27]:
# Crear mapas para tasas de no escolarización por nivel y media de géneros en dos filas
niveles_educacion = ['Preescolar', 'Primaria', 'Primer Ciclo Secundaria', 'Segundo Ciclo Secundaria']
num_filas = 2

fig = make_subplots(rows=num_filas, cols=len(niveles_educacion)//num_filas,
                    subplot_titles=[f'{nivel} - Media de Géneros' for nivel in niveles_educacion],
                    specs=[[{'type': 'choropleth'}] * (len(niveles_educacion)//num_filas) for _ in range(num_filas)])

colorbar_title = "Tasa de No Escolarización (Media)"
zmin = 0  # Valor mínimo para la barra de colores
zmax = 100  # Valor máximo para la barra de colores

for i, nivel in enumerate(niveles_educacion, start=1):
    # Calcular la media de las tasas para ambos géneros
    columna_varones = f'Tasa No Escolarización {nivel} (Varones)'
    columna_mujeres = f'Tasa No Escolarización {nivel} (Mujeres)'
    df[f'Tasa No Escolarización {nivel} (Media)'] = df[[columna_varones, columna_mujeres]].mean(axis=1)

    # Ordenar el DataFrame por la tasa media
    df_sorted = df.sort_values(by=f'Tasa No Escolarización {nivel} (Media)', ascending=False)

    # Obtener datos para el choropleth
    data = go.Choropleth(
        locations=df_sorted['Países y Zonas'],
        locationmode='country names',
        z=df_sorted[f'Tasa No Escolarización {nivel} (Media)'],
        colorscale='Viridis',
        colorbar=dict(title=colorbar_title),
        zmin=zmin,  # Establecer valor mínimo para la escala de colores
        zmax=zmax  # Establecer valor máximo para la escala de colores
    )

    # Añadir la traza al subgráfico correspondiente
    row = (i - 1) % num_filas + 1
    col = (i - 1) // num_filas + 1
    fig.add_trace(data, row=row, col=col)

fig.update_geos(projection_type='natural earth')
fig.update_layout(title_text="Tasas de No Escolarización por Nivel y Media de Géneros")
fig.show()

In [29]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Crear mapas para tasas de no escolarización por nivel y media de géneros en dos filas
niveles_educacion = ['Educación Primaria', 'Primer Ciclo Secundaria', 'Segundo Ciclo Secundaria']
num_filas = 3

fig = make_subplots(rows=num_filas, cols=len(niveles_educacion)//num_filas,
                    subplot_titles=[f'{nivel} - Media de Géneros' for nivel in niveles_educacion],
                    specs=[[{'type': 'choropleth'}] * (len(niveles_educacion)//num_filas) for _ in range(num_filas)])

colorbar_title = "Tasa Finalización (Media)"
zmin = 0  # Valor mínimo para la barra de colores
zmax = 100  # Valor máximo para la barra de colores

for i, nivel in enumerate(niveles_educacion, start=1):
    # Calcular la media de las tasas para ambos géneros
    columna_varones = f'Tasa Finalización {nivel} (Varones)'
    columna_mujeres = f'Tasa Finalización {nivel} (Mujeres)'
    df[f'Tasa Finalización {nivel} (Media)'] = df[[columna_varones, columna_mujeres]].mean(axis=1)

    # Ordenar el DataFrame por la tasa media
    df_sorted = df.sort_values(by=f'Tasa Finalización {nivel} (Media)', ascending=False)

    # Obtener datos para el choropleth
    data = go.Choropleth(
        locations=df_sorted['Países y Zonas'],
        locationmode='country names',
        z=df_sorted[f'Tasa Finalización {nivel} (Media)'],
        colorscale='Viridis',
        colorbar=dict(title=colorbar_title),
        zmin=zmin,  # Establecer valor mínimo para la escala de colores
        zmax=zmax  # Establecer valor máximo para la escala de colores
    )

      # Añadir la traza al subgráfico correspondiente
    row = (i - 1) % num_filas + 1
    col = ((i - 1) // num_filas) % (len(niveles_educacion) // num_filas) + 1
    fig.add_trace(data, row=row, col=col)

fig.update_geos(projection_type='natural earth')
fig.update_layout(title_text="Tasa Finalización por Nivel y Media de Géneros")
fig.show()